# Environment

In [246]:
import numpy as np
import csv
import pandas
from datetime import datetime
import json
import twitter
import tweepy
import time

In [228]:
info = dict()

# Enter your own consumer_key, consumer_secret, access_key and access_secret

info['CONSUMER_KEY'] = 'kzH6JHce26l6HhuMzHJWW8gDp'
info['CONSUMER_SECRET'] = 'QLwzCe6lNG1JriDGvn7gxGgrngtyWrPsXliuPNIlx0TGT4X7Rb'
info['ACCESS_KEY'] = '1094854919377637376-BL2OBuEtJCtQn6bzvEWcFiEzKn1pIf'
info['ACCESS_SECRET'] = 'oznzrzp4yNto96xFbyhZs4WYlJIYr5mn2fGr42gISasbi'

api = twitter.Api(consumer_key=info['CONSUMER_KEY'],
                      consumer_secret=info['CONSUMER_SECRET'],
                      access_token_key=info['ACCESS_KEY'],
                      access_token_secret=info['ACCESS_SECRET'])

auth = tweepy.OAuthHandler(info['CONSUMER_KEY'], info['CONSUMER_SECRET'])
auth.set_access_token(info['ACCESS_KEY'], info['ACCESS_SECRET'])

api = tweepy.API(auth)

In [190]:
# Variables

# Dictionary mapping day distance from June 04, 2010 to business day difference from June 04, 2010

start_date = datetime.strptime("2010-07-01-+0000", "%Y-%m-%d-%z")

# Data Prep

In [191]:
# Input: empty data matrix, csv reader object containing stock price information
# Output: data matrix containing:
# list of datetime objects marking 5 pm on the given day
# 0) number of days leading into this day from previous business day
# 1) closing stock price from previous day
# 2) change in stock price trom t-6 to t-5
# 3) change in stock price trom t-5 to t-4
# 4) change in stock price trom t-4 to t-3
# 5) change in stock price trom t-3 to t-2
# 6) change in stock price trom t-2 to t-1
# 8) change in stock price leading into current day

def stocks_data_fill(data):
    final_data = np.zeros((data.shape[0], 9))
    day_list = []
    
    i = 0
    j = 0

    while j < data.shape[0]:
        current_date = datetime.strptime(data[j, 0]+"-17:00:00-+0000", "%Y-%m-%d-%X-%z")
        #current_date = (current_date - start_date).days
        
        if (current_date - start_date).days >= 0:
            day_list.append(current_date)
            previous_date = datetime.strptime(data[j-1, 0]+"-17:00:00-+0000", "%Y-%m-%d-%X-%z")
            final_data[i,0] = (current_date - previous_date).days
            final_data[i,1] = data[j-1, 4]
            final_data[i, 2] = data[j-5, 4] - data[j-6, 4]
            final_data[i, 3] = data[j-4, 4] - data[j-5, 4]
            final_data[i, 4] = data[j-3, 4] - data[j-4, 4]
            final_data[i, 5] = data[j-2, 4] - data[j-3, 4]
            final_data[i, 6] = data[j-1, 4] - data[j-2, 4]
            final_data[i, 8] = data[j, 4] - data[j-1, 4]
            
            #final_data[i, 0] = int(data[j-1, 0].replace("-",""))
            #final_data[i, 1] = current_date
            #final_data[i, 2] = data[j-1, 4]
            #final_data[i, 3] = data[j-1, 4] - data[j-2, 4]
            #final_data[i, 6] = data[j, 4] - data[j-1, 4]
            i = i + 1
        j = j + 1
        
    return final_data, day_list

In [192]:
# Output: list of tweet timestamps and text
# Method: Manage rate limit by doing 100 tweets/call and implementing error handling. Auto-wait 15 mins before 
# making additional calls if rate limit has been reached

def make_tweet_matrix():
    with open('all_ids.json') as json_file:  
        tweet_ids = json.load(json_file)
        
    tweet_timestamps = []
    tweet_text = []
    
    i = 0
    iter_amt = 100
    
    while i < len(tweet_ids):
        try:
            if (i + iter_amt) < len(tweet_ids):
                tweets = tweet_ids[i:i + iter_amt]
            else:
                tweets = tweet_ids[i:len(tweet_ids)]
            tweets = api.statuses_lookup(tweets)
        except tweepy.error.TweepError:
            print("out of quota, waiting 15 minutes")
            time.sleep(60 * 15)
            continue
            
        for tweet in tweets:
            json_tweet = tweet._json
            tweet_time = json_tweet['created_at']
            tweet_date = datetime.strptime(tweet_time, "%a %b %d %X %z %Y")
            
            distance = (tweet_date - start_date).days
            if distance < -1:
                continue
            #tweets[0] = (tweet_date - start_date).days
            #tweets[1] = 
            #tweets[1] = tweet
            tweet_timestamps.append(tweet_date)
            tweet_text.append(json_tweet['text'])

        i = i + iter_amt
        
    return tweet_timestamps, tweet_text

In [193]:
# Input: data matrix filled in with first 3 features and the output, list containing timestamp objects for each row in data matrix,
# lists containing tweet timestamps and contents
# Output: data matrix containing:
# 1) number of days leading into this day from previous business day
# 2) closing stock price from previous day
# 3) change in stock price leading into previous day
# 4) Number of tweets between previous day closing time and current day closing time
# 5) change in stock price leading into current day
# list of datetime objects marking 5 pm on the given day
# list containing concatonated contents of tweets between previous day closing time and current day closing time
# Method: Iterate through twitter data and data matrix simultaneously, constructing matrix as we go

def tweets_data_fill(data, day_list, tweet_times, tweet_contents):
    i = 0
    j = 0
    
    tweet_text = []
    
    while i < len(data):
        tweets = []
        while tweet_times[j] < day_list[i]:
            tweets.append(tweet_contents[j])
            data[i, 7] = data[i, 7] + 1
            j = j + 1
        tweet_text.append(tweets)
        i = i + 1
    
    return data, day_list, tweet_text

In [194]:
# Use helper methods above to return the following information
# Output: data matrix containing:
# 1) number of days leading into this day from previous business day
# 2) closing stock price from previous day
# 3) change in stock price leading into previous day
# 4) Number of tweets between previous day closing time and current day closing time
# 5) change in stock price leading into current day
# list of datetime objects marking 5 pm on the given day
# list containing concatonated contents of tweets between previous day closing time and current day closing time

def prep_data():
    data_unformatted = pandas.read_csv('tesla_stock_prices.csv').as_matrix()
    data, day_list = stocks_data_fill(data_unformatted)
    data = data[~(data==0).all(1)]
    
    
    tweet_times, tweet_contents = make_tweet_matrix()
    my_order = [x for y, x in sorted(zip(tweet_times, range(len(tweet_times))))]
    tweet_times = [tweet_times[i] for i in my_order]
    tweet_contents = [tweet_contents[i] for i in my_order]
    
    return tweets_data_fill(data, day_list, tweet_times, tweet_contents)
    

# Write Data

In [195]:
# Write data matrix to a npy file
# Write timestamps for each row to a file
# Write tweets to a file

def write_data():
    data, day_list, tweet_contents = prep_data()
    day_list = [x.strftime("%Y-%m-%d-%X-%z") for x in day_list]
    
    np_path = "data/data.npy"
    day_path = "data/dates.json"
    tweet_contents_path = "data/tweets.json"
    
    
    np.save(np_path, data)
    
    json_day_contents = json.dumps(day_list)
    with open(day_path, 'w') as outfile:
        json.dump(json_day_contents, outfile)
    
    json_tweet_contents = json.dumps(tweet_contents)
    with open(tweet_contents_path, 'w') as outfile:
        json.dump(json_tweet_contents, outfile)

In [196]:
write_data()